In [1]:
from datetime import date
import awswrangler as wr
import pandas as pd

In [2]:
import boto3
boto3.setup_default_session(profile_name="quannguyen")

In [3]:
s3_path = "s3://upskills-landing-zone/secmaster/USSECMASTER_ALL20220525.XLS"

In [16]:
df = wr.s3.read_excel(s3_path, skiprows=4)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6145 entries, 0 to 6144
Data columns (total 50 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Symbol              6144 non-null   object 
 1   Name                6144 non-null   object 
 2   COMPANYNAME         6144 non-null   object 
 3   SYMBOL              6144 non-null   object 
 4   CUSIP               6144 non-null   object 
 5   SEDOL               6144 non-null   object 
 6   ISIN                6144 non-null   object 
 7   COUNTRYID           6144 non-null   float64
 8   EXCHANGECOUNTRY     6144 non-null   object 
 9   COUNTRY             6144 non-null   object 
 10  EXCHANGENAME        6144 non-null   object 
 11  FSSECTOR            6144 non-null   object 
 12  FSIND               6144 non-null   object 
 13  FSSECTORN           6144 non-null   float64
 14  FSINDN              6144 non-null   float64
 15  GICSSECTOR          6144 non-null   object 
 16  GICSIN

- Drop rows with full `nan`

In [18]:
df.dropna(how='all', inplace=True)

- Rename columns: 
    - 'SYMBOL' to 'SymbolCountry'

In [19]:
df.drop(columns=["Unnamed: 49"], inplace=True)

In [20]:
df.rename(columns={"SYMBOL": "SymbolCountry"}, inplace=True)

In [21]:
# convert float to int:
int_cols = ['COUNTRYID', 'FSSECTORN', 'FSINDN', 'ISONFF', 'RNDLOTS']
for col in int_cols:
    # convert null to float first and then to nullable Int32
    df[col] = df[col].astype(float).astype("Int32")

- There are 2 `DATETRADING` columns, so drop the one with more `nan` values
- convert column to `np.datetime` format

In [22]:
temp_df = df['DATETRADING.1'] if df['DATETRADING'].isna().sum() > df['DATETRADING.1'].isna().sum() else df['DATETRADING']

In [23]:
df.drop(columns=['DATETRADING', 'DATETRADING.1'], inplace=True)

In [24]:
df['DATETRADING'] = pd.to_datetime(temp_df, format="%Y%m%d")

- Convert `FISCALYEAREC` to `datetime` as well

In [25]:
df['FISCALYEAREC'] = pd.to_datetime(df['FISCALYEAREC'], format='%Y%m')

- Create partition columns

In [40]:
df['YEAR'] = df['DATETRADING'].dt.year
df['MONTH'] = df['DATETRADING'].dt.month

In [ ]:

# Write to S3
wr_response = wr.s3.to_parquet(
    df=df,
    path=os_input_s3_cleansed_layer,
    dataset=True,
    database=os_input_glue_catalog_db_name,
    table=os_input_glue_catalog_table_name,
    mode=os_input_write_data_operation,
    partition_cols=['COUNTRY', 'YEAR', 'MONTH', 'DAY'],
)

In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet('../data/aws_demo.snappy.parquet')

In [4]:
df.shape

(700, 47)